In [14]:
import cv2
import numpy as np


# 1.读入图片和模板
template = cv2.imread("../image/jinbi.png")
image_np = cv2.imread("../image/maliao.png")

# 2.对读取道德模板图及原图进行灰度化
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
image_np_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# 获取模板图的宽和高，方便后面进行绘图
h, w = template_gray.shape[:2]

# 3.进行模板匹配的操作
res = cv2.matchTemplate(image_np_gray, template_gray, cv2.TM_CCOEFF_NORMED)

# 4. 对返回的结果做进一步的处理
threshold =0.19

# 对某数组的数据进行判断，如果满足条件，就返回其对应的坐标
# 这里注意的是，他返回的是行和列的坐标。
# 对应到OpenCV重，就是某一个点的y和x的坐标
location = np.where(res > threshold)

# 5.对的得到的点的坐标进行反转处理，并在原图中框出来
for left_top in zip(*location[::-1]):
    right_bottom = (left_top[0] + w, left_top[1] + h)
    cv2.rectangle(image_np, left_top, right_bottom, (0, 0, 255))

# 6.显示模板匹配结果
cv2.imshow("image_np", image_np)
cv2.waitKey(0)

-1

In [ ]:
import cv2
import numpy as np


# 1.读入图片和模板
# template：作为匹配模板的小图
# image_np：待检测的大图（要在里边找模板）
template = cv2.imread("../image/jinbi.png")
image_np = cv2.imread("../image/maliao.png")

# 2.对读取到的模板图及原图进行灰度化
# 参数：
#   cv2.cvtColor(src, code)
#     src : 输入的 BGR 图像
#     code: 转换类型，这里为 BGR → GRAY
# 返回值：转为单通道的灰度图
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
image_np_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# 获取模板图的宽和高，方便后面绘制矩形使用
# shape 返回 (高h, 宽w)，这里取前两个维度
h, w = template_gray.shape[:2]

# 3.进行模板匹配的操作
# cv2.matchTemplate(image, templ, method)
# -------------------
# 模板匹配六种方法（method 参数）说明 + 原理 + 应用场景：
#
# 1. cv2.TM_SQDIFF
#    原理：计算模板与当前窗口的像素差异（差的平方求和）。差越小越相似。
#    特点：容易受亮度变化影响。
#    应用场景：模板和原图亮度一致、噪声少的场景。
#    最佳值：越小越好（完全一样时为0）。
#
# 2. cv2.TM_SQDIFF_NORMED
#    原理：在 SQDIFF 的基础上做归一化，让结果范围固定在 [0,1]。
#    特点：数值稳定，不会因为图片大小或像素强度不同导致偏移。
#    应用场景：不同分辨率的模板匹配；需要“越小越好”且稳定的情况。
#    最佳值：越小越好。
#
# 3. cv2.TM_CCORR
#    原理：模板与区域做“相关性计算”，简单理解为两个区域的像素是否随一起变化。
#    特点：模板亮度高时相关性结果可能偏大；受亮度影响明显。
#    应用场景：光照稳定、图像亮度一致的情况下。
#    最佳值：越大越好。
#
# 4. cv2.TM_CCORR_NORMED
#    原理：在 CCORR 基础上做归一化，结果范围固定在 [0,1]。
#    特点：减少图片强度影响，比 CCORR 稳定。
#    应用场景：需要强度鲁棒性，但光照变化不大的匹配任务。
#    最佳值：越大越好（通常大于 0.8 才可靠）。
#
# 5. cv2.TM_CCOEFF
#    原理：相关系数，计算模板与区域的“相似程度”，会自动忽略亮度偏移。
#    通俗理解：不看亮不亮，只看“形状像不像”。
#    特点：对整体亮度变化不敏感，比前几个更鲁棒。
#    应用场景：光照可能变化，但纹理结构不变的目标检测。
#    最佳值：越大越好。
#
# 6. cv2.TM_CCOEFF_NORMED  ← 你当前使用的方法
#    原理：在 CCOEFF 的基础上归一化，使结果范围 [-1,1]。
#    特点：
#       ✔ 最鲁棒，对光照、对比度变化影响最小
#       ✔ 匹配值 1 表示完全一致
#       ✔ 值在 0 附近表示毫不相关
#       ✔ 值为负表示相反结构
#    应用场景：最常用；光照变化明显、摄像头曝光不稳定、对比度不同的情况。
#    最佳值：越接近 1 越好。
#
# 总结（简单选择指南）：
#   - 图像亮度很稳定：CCORR、SQDIFF
#   - 光照变化大：使用 CCOEFF_NORMED（推荐）
#   - 模板亮度与目标亮度不同：CCOEFF 系列必用
#   - 想避免数值不稳定：选择带 NORMED 的版本
#
#
# 返回值：
#   res：每一个像素点处的匹配得分矩阵（数值越接近1越相似）
#
res = cv2.matchTemplate(
    image_np_gray,
    template_gray,
    cv2.TM_CCOEFF_NORMED,
)

# 4. 对返回的结果矩阵做进一步的处理
threshold = 0.19  # 设置匹配阈值，越大越严格

# np.where(res > threshold)
# 作用：找出所有匹配得分大于阈值的位置
# 返回值：两个数组 (y坐标数组, x坐标数组)
location = np.where(res > threshold)

# 5.对得到的坐标进行反转（因为返回 y在前，x在后）
# zip(*location[::-1])：
#   目的：将坐标组合成 (x, y) 形式，便于绘制矩形框
for left_top in zip(*location[::-1]):
    # 根据模板宽高计算右下角坐标
    right_bottom = (left_top[0] + w, left_top[1] + h)

    # 在原图中绘制矩形框住匹配到的区域
    # 参数：
    #   image_np   : 原图（彩色）
    #   left_top   : 左上角坐标
    #   right_bottom : 右下角坐标
    #   (0, 0, 255) : 红色
    #   2           : 线条粗细（默认是1，这里可增加清晰度）
    cv2.rectangle(image_np, left_top, right_bottom, (0, 0, 255))

# 6.显示模板匹配结果
# imshow 显示图像窗口
cv2.imshow("image_np", image_np)
cv2.waitKey(0)  # 等待按键关闭窗口